-------

## Setup

In [1]:
from Regressionizer import *
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

-------

## Generate input data

In [2]:
np.random.seed(0)
x = np.linspace(0, 2, 240)
y = np.sin(2 * np.pi * x) + np.random.normal(0, 0.4, x.shape)
data = np.column_stack((x, y))

Plot the generated data:

In [3]:
fig = px.scatter(x=data[:, 0], y=data[:, 1], labels={'x': 'X-axis', 'y': 'Y-axis'}, template='plotly_dark')
fig.show()

------

## Fit given functions

Define a list of functions:

In [4]:
funcs = [lambda x: 1, lambda x: x, lambda x: np.cos(x), lambda x: np.cos(3 * x), lambda x: np.cos(6 * x)]

In [5]:
def chebyshev_t_polynomials(n):
    if n == 0:
        return lambda x: 1
    elif n == 1:
        return lambda x: x
    else:
        T0 = lambda x: 1
        T1 = lambda x: x
        for i in range(2, n + 1):
            Tn = lambda x, T0=T0, T1=T1: 2 * x * T1(x) - T0(x)
            T0, T1 = T1, Tn
        return Tn

chebyshev_polynomials = [chebyshev_t_polynomials(i) for i in range(10)]

Define ***regression quantile*** probabilities:

In [6]:
probs = [0.1, 0.5, 0.9]

Perform quantile regression:

In [7]:
obj2 = (
    Regressionizer(data)
    .quantile_regression_fit(funcs=chebyshev_polynomials, probs=probs)
    .plot(title = "Chebyshev polynomials fit", template="plotly_dark")
)

Plot the obtained regression quantilies:

In [8]:
obj2.take_value().show()

-------

## Fit B-splines

Instead of coming-up with basis functions we can use B-spline basis:

In [9]:
obj = Regressionizer(data).quantile_regression(knots=5, probs=[0.2, 0.5, 0.8]).plot(title="B-splines fit", template="plotly_dark")

In [10]:
obj.take_value().show()

In [12]:
obj

Regressionizer object with data that has 240 records and 3 regression quantiles for [0.2, 0.5, 0.8].